In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 202501.xlsx to 202501.xlsx


In [ ]:
import pandas as pd

df = pd.read_excel('202501.xlsx')
print(df.head())

           Pump-Begin            Pump-End       MassResetTime  \
0 2025-01-10 10:53:43 2025-01-10 12:00:00 2025-01-10 12:00:00   
1 2025-01-10 12:00:00 2025-01-10 13:00:00 2025-01-10 13:00:00   
2 2025-01-10 13:00:00 2025-01-10 14:00:00 2025-01-10 14:00:00   
3 2025-01-10 14:00:01 2025-01-10 15:00:00 2025-01-10 15:00:00   
4 2025-01-10 15:00:00 2025-01-10 16:00:00 2025-01-10 16:00:00   

   Number-of-Split  Analysis-Id              Alarms           ElemError  \
0                0          NaN  0x0004000000000000  0x4000000000000000   
1                0       2917.0  0x0000000000000000  0x0000000000000000   
2                0       2918.0  0x0000000000000000  0x0000000000000000   
3                0       2919.0  0x0000000000000000  0x0000000000000000   
4                0       2920.0  0x0000000000000000  0x0000000000000000   

   Mass(ug)  Conc(ug/m3)  BP_Mass(ugC)  ...  Cd(ng/m3)  In(ng/m3)  Sn(ng/m3)  \
0     -22.1       -48.57           NaN  ...        NaN        NaN        NaN  

In [ ]:
import pandas as pd

df = pd.read_excel('202501.xlsx')

df['Pump-Begin'] = pd.to_datetime(df['Pump-Begin'])
df['Date'] = df['Pump-Begin'].dt.date

df.head()

,Pump-Begin,Pump-End,MassResetTime,Number-of-Split,Analysis-Id,Alarms,ElemError,Mass(ug),Conc(ug/m3),BP_Mass(ugC),...,In(ng/m3),Sn(ng/m3),Sb(ng/m3),Te(ng/m3),Cs(ng/m3),Ba(ng/m3),Ce(ng/m3),Pb(ng/m3),Bi(ng/m3),Date
0,2025-01-10 10:53:43,2025-01-10 12:00:00,2025-01-10 12:00:00,0,NaN,0x0004000000000000,0x4000000000000000,-22.1,-48.57,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-01-10
1,2025-01-10 12:00:00,2025-01-10 13:00:00,2025-01-10 13:00:00,0,2917.0,0x0000000000000000,0x0000000000000000,-19.7,-19.84,NaN,...,57.61,30.11,29.99,5.94,-1.80,0.0,-0.43,16.83,6.62,2025-01-10
2,2025-01-10 13:00:00,2025-01-10 14:00:00,2025-01-10 14:00:00,0,2918.0,0x0000000000000000,0x0000000000000000,10.1,10.20,NaN,...,69.71,47.75,-11.84,28.56,-1.89,0.0,-0.96,13.57,11.04,2025-01-10
3,2025-01-10 14:00:01,2025-01-10 15:00:00,2025-01-10 15:00:00,0,2919.0,0x0000000000000000,0x0000000000000000,6.9,6.90,NaN,...,54.34,6.92,28.12,47.94,-0.71,0.0,-0.13,16.47,8.02,2025-01-10
4,2025-01-10 15:00:00,2025-01-10 16:00:00,2025-01-10 16:00:00,0,2920.0,0x0000000000000000,0x0000000000000000,6.2,6.26,NaN,...,55.92,23.70,24.88,23.03,-1.84,0.0,-0.71,2.43,7.10,2025-01-10


In [ ]:
df = df.drop(columns=['Number-of-Split', 'Analysis-Id', 'Alarms', 'ElemError', 'BP_Mass(ugC)', 'BP_Conc(ugC/m3)'])

print(df.head())

           Pump-Begin            Pump-End       MassResetTime  Mass(ug)  \
0 2025-01-10 10:53:43 2025-01-10 12:00:00 2025-01-10 12:00:00     -22.1   
1 2025-01-10 12:00:00 2025-01-10 13:00:00 2025-01-10 13:00:00     -19.7   
2 2025-01-10 13:00:00 2025-01-10 14:00:00 2025-01-10 14:00:00      10.1   
3 2025-01-10 14:00:01 2025-01-10 15:00:00 2025-01-10 15:00:00       6.9   
4 2025-01-10 15:00:00 2025-01-10 16:00:00 2025-01-10 16:00:00       6.2   

   Conc(ug/m3)  Al(ng/m3)  Si(ng/m3)  S(ng/m3)  K(ng/m3)  Ca(ng/m3)  ...  \
0       -48.57        NaN        NaN       NaN       NaN        NaN  ...   
1       -19.84     793.97     834.70   2338.79    358.10     155.00  ...   
2        10.20    1004.34     727.61   1361.91    281.32     124.75  ...   
3         6.90     477.28     687.55   1387.49    267.52     128.43  ...   
4         6.26     465.45     717.28   1441.36    266.17     138.57  ...   

   In(ng/m3)  Sn(ng/m3)  Sb(ng/m3)  Te(ng/m3)  Cs(ng/m3)  Ba(ng/m3)  \
0        NaN        N

In [ ]:
import numpy as np

numeric_cols = df.select_dtypes(include=np.number).columns
df_cleaned = df[(df[numeric_cols] >= 0).all(axis=1)]

print(df_cleaned.head())

             Pump-Begin            Pump-End       MassResetTime  Mass(ug)  \
412 2025-01-27 15:00:00 2025-01-27 16:00:00 2025-01-27 16:00:00      23.2   

     Conc(ug/m3)  Al(ng/m3)  Si(ng/m3)  S(ng/m3)  K(ng/m3)  Ca(ng/m3)  ...  \
412        23.25    1922.71     9506.0    2784.1    365.05     716.64  ...   

     In(ng/m3)  Sn(ng/m3)  Sb(ng/m3)  Te(ng/m3)  Cs(ng/m3)  Ba(ng/m3)  \
412      51.11      26.05      18.14      30.56      29.28        0.0   

     Ce(ng/m3)  Pb(ng/m3)  Bi(ng/m3)        Date  
412      21.73       7.39       6.27  2025-01-27  

[1 rows x 41 columns]
